# Example 1: class Beam from ray-tracing codes output

**Goal:**  this example shows how to create and manipulate a 'standard beam' from common ray-tracing codes outputs (SHADOW3, SHADOW4 or PyOptiX). 


In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'CECILL-2.1'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '2025.11.07'
__changed__ = '2025.11.10'

import barc4beams as b4b

## Conversion to **barc4beams.Beams()**

> - **SHADOW4 beam**: 
>
>         beam_s4 = b4b.Beam(s4_beam, code="s4")
>
> - **SHADOW3 beam**: 
>
>        beam_s3 = b4b.Beam(s3_beam, code="s3")
>
> - **PyOptiX DataFrame** from ```OpticalElement.get_diagram(...)``` or ```OpticalElement.get_impacts(...)``` : 
>
>        beam_pyoptix = b4b.Beam(pyoptix_df, code="pyoptix")


## Toroidal mirror focusing

### 1) Standardize & wrap

In [ ]:
# path to your exported file
path = "./beams/s4_ideal_toroid_focusing.h5"

# read the file and validate schema
std_beam = b4b.read_beam(path)
b4b.schema.validate_beam(std_beam)

# Beam object
beam = b4b.Beam(std_beam)

# quick peek at the standard beam columns
beam.df.head()

### 2) Quick statistics

In [ ]:
stats = beam.stats(verbose=True)
stats  # also stored for saving later

#### Understanding the beam moments

When we print beam statistics, we get a set of **moments** describing the overall *shape* of the beam’s spatial or angular distributions:

- **Mean** — the *average position* of the rays.  
  It shows where the beam is centered.  
  A non-zero mean means the beam is offset from the optical axis.

- **Standard deviation (σ)** — the *spread* or *width* of the beam around its mean.  
  It represents the RMS size or divergence.  
  A smaller σ means a tighter, better-focused beam.

- **Skewness** — the *asymmetry* of the distribution.  
  If positive, the beam has a longer “tail” on the right side;  
  if negative, the tail is on the left.  
  A symmetric beam has skewness ≈ 0.

- **Kurtosis** — the *peakedness* of the profile.  
  A Gaussian beam has kurtosis ≈ 0 (in the “excess” convention).  
  Positive kurtosis means a sharper peak and heavier tails;  
  negative values indicate a flatter, more top-hat-like profile.

Together, these moments describe how **centered**, **broad**, **asymmetric**, or **peaked** the beam is — giving a quantitative summary of its shape before any plotting.


### 3) Plots
We generate the full set of available plots once. You can adjust ranges if desired. if ```path``` is provided, the image is saved.

In [ ]:
# Beam profile (X vs Y)
beam.plot_beam(mode="hist2d", x_range=[-30, 60], y_range=[-20, 20], aspect_ratio=True, color=1, path=None)

In [ ]:
# Divergence (dX vs dY)
beam.plot_divergence(mode="hist2d", aspect_ratio=False, color=2, path=None)

In [ ]:
# Phase-space for both planes
beam.plot_phase_space(direction="both", mode="scatter", aspect_ratio=False, color=3, path=None)

### Understanding phase-space plots

Phase-space plots show the relationship between **position** and **angle** (X vs dX, or Y vs dY) for every ray.  
They are the most direct way to visualize the *optical state* of the beam — whether it’s parallel, focusing, or diverging.

- **Ideal Gaussian beam (no aberrations):**
  - The cloud of points forms a smooth **ellipse** in phase space.
  - The ellipse orientation tells you whether the beam is converging, diverging, or collimated.

- **Focused beam (at or near focus):**
  - The ellipse is **upright and narrow** — position and angle are largely uncorrelated.
  - The beam waist (minimum size) corresponds to the point where this ellipse is smallest.

- **Diverging beam (after focus):**
  - The ellipse tilts **upward** — rays on one side of the axis leave with angles pointing outward.
  - Position and angle are **positively correlated**.

- **Focusing beam (before focus):**
  - The ellipse tilts **downward** — rays on opposite sides of the axis are converging toward the focal point.
  - Position and angle are **negatively correlated**.

- **Parallel (collimated) beam:**
  - The ellipse becomes almost **flat and horizontal** — positions vary, but angles are all nearly zero.

Aberrations, astigmatism, or coma distort this ellipse — turning it from a clean, tilted oval into irregular or curved shapes.
A well-behaved, diffraction-limited beam always preserves this elliptical geometry across propagation.

In [ ]:
# Energy distribution
beam.plot_energy(path=None)

In [ ]:
# Energy vs intensity (helpful for polychromatic runs)
beam.plot_energy_vs_intensity(mode="hist2d", color=3, path=None)

### 4A) Propagate by 1.0 m and re-plot footprint

In [ ]:
beam_1m = beam.propagate(1.0)
stats = beam_1m.stats(verbose=True)
beam_1m.plot_beam(mode="hist2d", aspect_ratio=True, color=1, path=None)

### 4B) The beam can be plotted directly at 1 meter from the source without creating a new Beam object

In [ ]:
beam.plot_beam(mode="hist2d", aspect_ratio=True, color=1, path=None, z_offset=1.0)

## 5) Caustic (X/Z and Y/Z) with top standard deviation panel

In [ ]:
beam.plot_caustic(which="both", top_stat="std", n_points=501, start=-0.20, finish=0.20, path=None)

### 6) Save beam and stats, then reload

In [ ]:
b4b.save_beam(beam.df, "./beams/example_01_beam.h5")
b4b.save_json_stats(stats, "example_01_beam_stats.json")

beam2 = b4b.Beam.from_h5("./beams/example_01_beam.h5")
_ = beam2.stats(verbose=True)
print("\n\n>>>>> Reloaded beam OK.")